In [1]:
#Ejercicio 6:
# Matrix treatment:
def nn2na(NN):
    # Get every location where exist an arc:
    idxs = np.argwhere(NN)
    # Preallocate NA matrix, dimension is (nodes, arcs)
    NA = np.zeros([NN.shape[0], idxs.shape[0]]).astype(int)
    C = np.zeros(NA.shape[1])
    # Loop in every arc, complete from (1) to (-1)
    for i, arc in enumerate(idxs):
        # Node arc:
        NA[arc[0], i] = 1 # From
        NA[arc[1], i] = -1 # To

    arc_idxs = [(arc[0], arc[1]) for arc in idxs]

    return NA, arc_idxs



# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

# Maximum flow Utils
def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]

import numpy as np
from scipy.optimize import linprog
from basic_utils import nn2na, get_usage_string, get_min_cut

# IMPORT THE DATA:
# NN = np.array([[0, 0, 0, 0, 1, 1],
#                [0, 0, 0, 0, 1, 1],
#                [0, 0, 0, 0, 1, 1],
#                [0, 0, 0, 0, 0, 0],
#                [0, 0, 0, 0, 0, 0],
#                [0, 0, 0, 0, 0, 0]])esta es tu matriz daniel aca tenia una fila de mas osea
#son 5 nodos y tenias 6 asi que te quite una

NN = np.array([[ 0, 0, 0, 1, 1],
               [0, 0, 0, 1, 1],
               [0, 0, 0, 1, 1],
               [0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0]])


# DATA MANIPULATION:
Aeq, arc_idxs = nn2na(NN)

C = np.array([10, 20, 10, 10, 10, 30])

beq = np.array([10, 20, 15, -25, -20])

max_q = [None, None, None, None, None, None]

bounds = tuple([(0, max_q[arcs]) for arcs in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n '
      'A_eq Node-Arc matrix:\n%s \n'
      'b_eq demand-supply vector: %s \n'
      'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds))

# OPTIMIZE:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

# GET THE SOLUTION:
usage = get_usage_string(arc_idxs, res.x.astype(int), max_q)
min_cut = get_min_cut(arc_idxs, res.x, np.array(max_q))
max_flow = res.fun * -1
print('## Results ## \n'
      'The usage of each arc will be (from, to): %s \n'
      'The arcs that produce the minimum cut (from, to): %s \n'
      'The maximum flow will be: %0.2f ' % (usage, min_cut , max_flow))



## Optimizer inputs ## 
Cost vector: [10 20 10 10 10 30] 
 A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0]
 [ 0  0  1  1  0  0]
 [ 0  0  0  0  1  1]
 [-1  0 -1  0 -1  0]
 [ 0 -1  0 -1  0 -1]] 
b_eq demand-supply vector: [ 10  20  15 -25 -20] 
Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 

## Results ## 
The usage of each arc will be (from, to): {(0, 3): '10/None', (0, 4): '0/None', (1, 3): '0/None', (1, 4): '20/None', (2, 3): '15/None', (2, 4): '0/None'} 
The arcs that produce the minimum cut (from, to): [] 
The maximum flow will be: -450.00 


<ipython-input-1-62316e4c7d4a>:77: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
